# AsterixDB

In [81]:
from asterixdb.asterixdb import AsterixConnection
from glob import glob
import json
import os
from datetime import datetime
import numpy as np

## Connect to Asterix

Establish connection to AsterixDB:

(using https://github.com/j-goldsmith/asterixdb-python)

In [82]:
con = AsterixConnection(server='http://localhost', port=19002)

Example query:

In [17]:
# response = con.query('''
#     USE TinySocial;

#     SELECT VALUE user
#     FROM GleambookUsers user
#     WHERE user.id = 8;''')

# response.results

[{'id': 8,
  'alias': 'Nila',
  'name': 'NilaMilliron',
  'userSince': '2008-01-01T10:10:00.000Z',
  'friendIds': [3],
  'employment': [{'organizationName': 'Plexlane', 'startDate': '2010-02-28'}]}]

Clear dataverse if it already exists(!) and create new

In [18]:
# response = con.query('''
#     DROP DATAVERSE FactMap IF EXISTS;
#     CREATE DATAVERSE FactMap;
#     ''')

---

## Reddit

First create subset of only 2019 submissions for experimentation:

In [86]:
rt_paths = sorted(glob("/Users/ageil/Github/FactMap/Data/reddit/2019/*.json"))
combined = ""

for p in rt_paths:
    combined += 'localhost://' + p + ','
    
combined = combined[:-1]  # remove last ','

In [87]:
q = '''
    USE FactMap;
    
    DROP TYPE submissionTypeTemp IF EXISTS;
    CREATE TYPE submissionTypeTemp as {{
        uid: uuid
    }};
    
    DROP DATASET PostsTemp IF EXISTS;
    CREATE DATASET PostsTemp(submissionTypeTemp)
        PRIMARY KEY uid AUTOGENERATED;
                
    LOAD DATASET PostsTemp
    USING localfs (("path"="{0}"),("format"="json"));
    '''.format(combined)

In [54]:
response = con.query(q)

Cast to intended data format and clean up temp table:

In [21]:
response = con.query('''
    USE FactMap;

    DROP DATASET posts2019 IF EXISTS;
    
    CREATE TYPE SubmissionType as {
        id: string
    };
    CREATE DATASET posts2019(SubmissionType)
        PRIMARY KEY id;

    INSERT INTO posts2019
    SELECT 
    id,
    datetime_from_unix_time_in_secs(int(created_utc)) as created_utc,
    subreddit_id,
    subreddit,
    author,
    domain,
    int(score) as score,
    int(num_comments) as num_comments,
    title,
    url
    FROM PostsTemp p;
    
    DROP DATASET PostsTemp IF EXISTS;
    DROP TYPE SubmissionTypeTemp IF EXISTS;
    ''')

Now load all reddit submissions with direct load (note BigQuery gives `int` as `string` so have to cast):

In [88]:
rt_paths = sorted(glob("/Users/ageil/Github/FactMap/Data/reddit/new/*.json"))
combined = ""

for p in rt_paths:
    combined += 'localhost://' + p + ','
    
combined = combined[:-1]  # remove last ,

In [89]:
q = '''
    USE FactMap;
    
    DROP TYPE submissionTypeTemp IF EXISTS;
    CREATE TYPE submissionTypeTemp as {{
        uid: uuid
    }};
    
    DROP DATASET PostsTemp IF EXISTS;
    CREATE DATASET PostsTemp(submissionTypeTemp)
        PRIMARY KEY uid AUTOGENERATED;
                
    LOAD DATASET PostsTemp
    USING localfs (("path"="{0}"),("format"="json"));
    '''.format(combined)

In [59]:
response = con.query(q)

Cast to intended data format and clean up temp table:

In [63]:
response = con.query('''
    USE FactMap;

    DROP DATASET posts IF EXISTS;

    CREATE DATASET posts(SubmissionType)
        PRIMARY KEY id;

    UPSERT INTO posts
    SELECT 
        id,
        datetime_from_unix_time_in_secs(int(created_utc)) as created_utc,
        subreddit_id,
        subreddit,
        author,
        domain,
        int(score) as score,
        int(num_comments) as num_comments,
        title,
        url
    FROM PostsTemp p;
    
    DROP DATASET PostsTemp IF EXISTS;
    ''')

note: use `UPSERT INTO` to protect against duplicate ids

---

## ClaimReview

Load all claims, generate uid:

In [46]:
response = con.query('''
    USE FactMap;
    
    DROP TYPE ReviewsType IF EXISTS;
    CREATE TYPE ReviewsType as {
        uid: uuid
    };
    
    DROP DATASET claims IF EXISTS;
    CREATE DATASET claims(ReviewsType)
        PRIMARY KEY uid AUTOGENERATED;
                
    LOAD DATASET claims
    USING localfs (("path"="localhost:///Users/ageil/Github/FactMap/Data/claimreviews/claims_2020_combined.json"),("format"="json"));
    ''')

Clean up formatting:

In [47]:
response = con.query('''
    USE FactMap;
    
    CREATE DATASET reviews(ReviewsType)
        PRIMARY KEY uid;

    INSERT INTO reviews
    SELECT 
        uid,
        reviewUrl,
        claimReviewed,
        countries,
        claimReviewed_en,
        datetime_from_unix_time_in_secs(claimDate) as claimDate,
        datetime_from_unix_time_in_secs(reviewDate) as reviewDate,
        reviewAuthor,
        reviewRating,
        claimAuthor,
        tagsRaw,
        tagsNamed,
        reviewTitle
    FROM claims c;
    
    DROP DATASET claims IF EXISTS;
    ''')    

Save subset with valid/invalid ratings:

In [48]:
import pickle

In [50]:
rated = con.query('''
    USE FactMap;
    
    SELECT r.*
    FROM reviews r
    WHERE r.reviewRating.bestRating >= r.reviewRating.ratingValue
    AND r.reviewRating.ratingValue >= r.reviewRating.worstRating
    AND r.reviewRating.bestRating > r.reviewRating.worstRating;
''').results

with open('/Users/ageil/Github/FactMap/RNN/data/rated_2020.pickle', 'wb') as f:
    pickle.dump(rated, f, pickle.HIGHEST_PROTOCOL)
    
len(rated)

64798

In [51]:
unrated = con.query('''
    USE FactMap;
    
    SELECT r.*
    FROM reviews r
    WHERE NOT (r.reviewRating.bestRating >= r.reviewRating.ratingValue
    AND r.reviewRating.ratingValue >= r.reviewRating.worstRating
    AND r.reviewRating.bestRating > r.reviewRating.worstRating)
    OR is_null(r.reviewRating.ratingValue)
    OR is_null(r.reviewRating.worstRating)
    OR is_null(r.reviewRating.bestRating);
''').results

with open('/Users/ageil/Github/FactMap/RNN/data/unrated.pickle', 'wb') as f:
    pickle.dump(unrated, f, pickle.HIGHEST_PROTOCOL)

len(unrated)

86227

## Word vectors (FastText)

NOTE: See vec2json2asterix.py for more efficient pipeline and split original file into 4 using:

```
split -l 500000 crawl_300d_2M.vec crawl_300d_2M
```

Then run vec2json.py on the resulting files (in parallel), and use json2asterix.py to push them to the database (in parallel).

```
USE FactMap;

DROP DATASET fasttext IF EXISTS;
DROP TYPE EmbeddingType IF EXISTS;

CREATE TYPE EmbeddingType AS {
    word: string,
    vector: [float]
};

CREATE DATASET fasttext(EmbeddingType)
    PRIMARY KEY word;

INSERT INTO fasttext
    ([
        {"word": "hello", "vector": [0.0, 0.1]}
    ]);

SELECT *
FROM fasttext;
```

Using Facebook Research's 300-dimensional vectors for 2M words trained on Common Crawl:
https://fasttext.cc/docs/en/english-vectors.html

Importantly, these were not trained on Wikipedia but on socially 'flawed' words (like our own data)!

In [3]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [4]:
fname = "/Users/anders1991/Github/FactMap/data/fasttext/crawl-300d-2M.vec"

vectors = load_vectors(fname) 

Initialize dataset:

In [5]:
response = con.query('''
    USE FactMap;

    DROP DATASET fasttext IF EXISTS;
    DROP TYPE EmbeddingType IF EXISTS;

    CREATE TYPE EmbeddingType AS {
        word: string,
        vector: [float]
    };

    CREATE DATASET fasttext(EmbeddingType)
        PRIMARY KEY word;
    ''')

Insert word embeddings into dataset:

In [3]:
failed = []

for w, v in vectors.items():
    try:
        word = w
        vec = *v,
        response = con.query('''
            USE FactMap;
            
            INSERT INTO fasttext
            ([{{"word":"{0}", "vector":{1}}}])
        '''.format(word, list(vec)))
    except:
        failed.append(w)

# print("Failed word inserts:", len(failed))

---

## Joining

### Hard join (claims = news articles)

Join on full fake news source URL.

In [14]:
response = con.query('''
    USE FactMap;

    SET `compiler.joinmemory` "128MB";

    DROP DATASET urljoin IF EXISTS;
    DROP TYPE PostReviewType IF EXISTS;

    CREATE TYPE PostReviewType as {
        r: ReviewsType,
        p: SubmissionType
    };

    CREATE DATASET urljoin(PostReviewType)
        PRIMARY KEY r.uid, p.id;

    INSERT INTO urljoin
    SELECT *
    FROM posts p, reviews r
    WHERE r.claimAuthor.claimURL = p.url;
    ''')

In [92]:
response = con.query('''
    USE FactMap;
    
    SELECT u.*
    FROM urljoin u;
    ''')

print('Number of matches:', len(response.results))

Number of matches: 19396


was 14325 pre 2020 update

So how many unique claimreviews are represented?

In [93]:
response = con.query('''
    USE FactMap;

    SELECT count(distinct r.uid) as c
    FROM urljoin u
    LIMIT 1;
    ''')
print('Number of unique claims:', response.results[0]['c'])

Number of unique claims: 2623


1652 unique claims are posted across the 14325 reddit submissions.

And how many of the ratings are numerically valid/invalid?

In [94]:
response = con.query('''
    USE FactMap;

    SELECT count(distinct r.uid) as c
    FROM urljoin u
    WHERE 
    (r.reviewRating.worstRating < r.reviewRating.bestRating 
    AND
    r.reviewRating.worstRating <= r.reviewRating.ratingValue
    AND
    r.reviewRating.ratingValue <= r.reviewRating.bestRating)
    LIMIT 1;
    ''')
print('Number of unique, valid numerical ratings:', response.results[0]['c'])

Number of unique, valid numerical ratings: 1171


was 923 pre 2020 update

In [95]:
response = con.query('''
    USE FactMap;

    SELECT count(distinct r.uid) as c
    FROM urljoin u
    WHERE 
    NOT
    (r.reviewRating.worstRating < r.reviewRating.bestRating 
    AND
    r.reviewRating.worstRating <= r.reviewRating.ratingValue
    AND
    r.reviewRating.ratingValue <= r.reviewRating.bestRating)
    LIMIT 1;
    ''')
print('Number of unique, invalid numerical ratings:', response.results[0]['c'])

Number of unique, invalid numerical ratings: 1203


was 723 pre 2020 update

### Hard join (review articles)

In [105]:
response = con.query('''
    USE FactMap;

    SET `compiler.joinmemory` "128MB";

    DROP DATASET facturljoin IF EXISTS;
    
    CREATE DATASET facturljoin(PostReviewType)
        PRIMARY KEY r.uid, p.id;

    INSERT INTO facturljoin
    SELECT *
    FROM posts p, reviews r
    WHERE r.reviewUrl = p.url;
    ''')

In [106]:
response = con.query('''
    USE FactMap;
    
    SELECT u.*
    FROM facturljoin u;
    ''')

print('Number of matches:', len(response.results))

Number of matches: 27213


was 19224 pre 2020 update

In [107]:
response = con.query('''
    USE FactMap;

    SELECT COUNT(DISTINCT r.uid) unique_claims
    FROM facturljoin f;
    ''')

print('Number of unique claims', response.results[0]['unique_claims'])

Number of unique claims 12918


was 8636 pre 2020 update

So there's actually more corrected news posted to reddit than fake news!

In [71]:
response = con.query('''
    USE FactMap;

    SELECT COUNT(r.uid) as c, g
    FROM facturljoin f
    GROUP BY r.uid
    GROUP AS g
    ORDER BY c desc
    LIMIT 5;
    ''').results

In [72]:
for i in response:
    print("Number of cross-posts:", i['c'])
    
    subs = [p['f']['p']['subreddit'] for p in i['g']]
    print("Subreddits:\n", subs)
    print()

Number of cross-posts: 36
Subreddits:
 ['politics', 'uspolitics', 'worldpolitics', 'DieOff', 'DieOff', 'redacted', 'LyingTrump', 'moderatepolitics', 'Libertarian', 'Impeach_Trump', 'Laserlike_Hodgepodge', 'topofreddit', 'CringeAnarchy', 'Fuck45', 'worldpolitics', 'worldnews', 'esist', 'gogopgo', 'democrats', 'u_the_foreign_code', 'u_the_economy_club', 'POLITIC', 'TrumptASStic', 'POLITIC', 'u_talesofafallenforest', 'EnoughTrumpSpam', 'POLITIC', 'politics', 'NoShitSherlock', 'POLITIC', 'TrumpTellsTales', 'redacted', 'POLITIC', 'POLITIC', 'chomsky', 'LeftCentral']

Number of cross-posts: 36
Subreddits:
 ['politics', 'uspolitics', 'worldpolitics', 'DieOff', 'DieOff', 'redacted', 'LyingTrump', 'moderatepolitics', 'Libertarian', 'Impeach_Trump', 'Laserlike_Hodgepodge', 'topofreddit', 'CringeAnarchy', 'Fuck45', 'worldpolitics', 'worldnews', 'esist', 'gogopgo', 'democrats', 'u_the_foreign_code', 'u_the_economy_club', 'POLITIC', 'TrumptASStic', 'POLITIC', 'u_talesofafallenforest', 'EnoughTrumpS

On further inspection, there's no need for further fuzzy matching on the review matches.

### Hard join w/ fuzzy join on Twitter/Wikipedia (claims)

More refined join, using hard URL join only except for wikipedia and twitter posts, since these URLs to these sites are not claim specific and may refer to multiple tweets/sections.

Additional requirements for Twitter/Wikipedia posts include:
- The post domain must contain either `wikipedia` or `twitter`.
- Minimum caption length is 15 characters (cannot meaningfully distinguish claims below this threshold).
- Similar-length claim and post titles must have at least 20% word tokens in common.
    - Similar-length is defined as the difference in length between the two titles does not exceed 20% of the shortest title.
- Different-length claim and post titles must be contained within one or the other, changing at most 50% of the characters to generate a perfect match to the containing title.
    - Different-length is defined as the difference in length between the two titles must be at least 20% of the shortest title.

Create table for fuzzy URL join:

In [96]:
response = con.query('''
    USE FactMap;

    SET `compiler.joinmemory` "128MB";

    DROP DATASET fuzzyurljoin IF EXISTS;
    
    CREATE DATASET fuzzyurljoin(PostReviewType)
        PRIMARY KEY r.uid, p.id;
    ''')

Get URL matches with same-length review and post titles (Jaccard similarity > 20%):

In [97]:
response = con.query('''
        USE FactMap;
        
        INSERT INTO fuzzyurljoin
        SELECT u.*
        FROM urljoin u
        WHERE
            (similarity_jaccard(word_tokens(lower(p.title)), word_tokens(lower(r.claimReviewed))) > 0.20
            OR similarity_jaccard(word_tokens(lower(p.title)), word_tokens(lower(r.claimReviewed_en))) > 0.20)
            AND (abs(length(r.claimReviewed) - length(p.title)) <= 
                (array_min([length(r.claimReviewed), length(p.title)])) * 0.2)
            AND (array_min([length(r.claimReviewed), length(p.title)]) > 15)
            AND (contains(p.domain, "wikipedia") OR contains(p.domain, "twitter"));
        ''')

= 227 matches added.

= 260 matches post 2020 update

Get URL matches with different-length review and post titles (edit distance > 0.5 * min(review or post title length)):

In [98]:
response = con.query('''
    USE FactMap;
    
    INSERT INTO fuzzyurljoin
    SELECT u.*
    FROM urljoin u
    WHERE
            (
                (edit_distance_contains(lower(p.title), lower(r.claimReviewed), length(r.claimReviewed) * 0.5)[0] 
                    OR edit_distance_contains(lower(r.claimReviewed), lower(p.title), length(p.title) * 0.5)[0])
                OR
                (edit_distance_contains(lower(p.title), lower(r.claimReviewed_en), length(r.claimReviewed_en) * 0.5)[0] 
                    OR edit_distance_contains(lower(r.claimReviewed_en), lower(p.title), length(p.title) * 0.5)[0])
            )
            AND (abs(length(r.claimReviewed) - length(p.title)) > 
                (array_min([length(r.claimReviewed), length(p.title)])) * 0.2)
            AND (array_min([length(r.claimReviewed), length(p.title)]) > 15)
            AND (contains(p.domain, "wikipedia") OR contains(p.domain, "twitter"));
    ''')

= 1166 matches added.

= 1407 matches added post 2020 update

Add back in all the other url-joined pairs that are not linked to wikipedia or twitter:

In [99]:
response = con.query('''
    USE FactMap;
    
    INSERT INTO fuzzyurljoin
    SELECT u.*
    FROM urljoin u
    WHERE NOT (contains(p.domain, "wikipedia") OR contains(p.domain, "twitter"));
    ''')

= 6809 matches added.

= 10,113 matches added post 2020 update

Count total number of (fuzzy) url joined matches:

In [100]:
response = con.query('''
    USE FactMap;
    
    SELECT COUNT(*) as total
    FROM fuzzyurljoin u;
    ''')

print('Number of matches:', response.results[0]['total'])

Number of matches: 12079


From 14325 matches to 8202. A reduction of 6323 false matches.

Post 2020 update: From 19068 matches to 11780. A reduction of 7288 false matches.

Post 2020 full update: From 19396 matches to 12079. A reduction of 7317 false matches.

In [102]:
response = con.query('''
    USE FactMap;

    SELECT COUNT(DISTINCT r.uid) unique_claims
    FROM fuzzyurljoin f;
    ''')

print('Number of unique claims', response.results[0]['unique_claims'])

Number of unique claims 2289


From 1652 unique claims to 1378. A reduction of 274 falsely matched claims.

Post 2020 update: From 2563 unique claims to 2233. A reduction of 330 falsely matched claims.

Post 2020 full update: From 2623 unique claims to 2289. A reduction of 334 falsely matched claims.

In [103]:
response = con.query('''
    USE FactMap;

    SELECT count(distinct r.uid) as c
    FROM fuzzyurljoin u
    WHERE 
    (r.reviewRating.worstRating < r.reviewRating.bestRating 
    AND
    r.reviewRating.worstRating <= r.reviewRating.ratingValue
    AND
    r.reviewRating.ratingValue <= r.reviewRating.bestRating)
    LIMIT 1;
    ''')
print('Number of unique, valid numerical ratings:', response.results[0]['c'])

Number of unique, valid numerical ratings: 960


In [104]:
response = con.query('''
    USE FactMap;

    SELECT count(distinct r.uid) as c
    FROM fuzzyurljoin u
    WHERE 
    NOT
    (r.reviewRating.worstRating < r.reviewRating.bestRating 
    AND
    r.reviewRating.worstRating <= r.reviewRating.ratingValue
    AND
    r.reviewRating.ratingValue <= r.reviewRating.bestRating)
    LIMIT 1;
    ''')
print('Number of unique, invalid numerical ratings:', response.results[0]['c'])

Number of unique, invalid numerical ratings: 1098


----

Helpful queries:

In [ ]:
# r = con.query('''
#     use FactMap;
#     SELECT VALUE ds FROM Metadata.`Dataset` ds WHERE DataverseName = 'FactMap';
#     SELECT VALUE ds FROM Metadata.`Dataset` ds;
#     SELECT VALUE ix FROM Metadata.`Index` ix;''')

# r.results